# User-defined types 

As we have started to see, the basic types in Julia, such as `Complex`, are defined in Julia code.
They are treated on the same footing as types defined by the user.

Composite types in Julia are just boxes that hold data and provide constructor functions for making new objects of that type. They *do not contain methods (functions)* (as would be the case in many other object-oriented languages); rather, the methods are defined using multiple dispatch.

## Example: Fixed-size vectors 

Currently, there are no fixed-size vectors in base Julia (although these are available, for example, in the `ImmutableArrays` and `FixedSizeVectors` packages). For many applications, fixed-size vectors are useful, for example for representing positions of particles moving in 2D or 3D space. We can define our own type that may prove to be more efficient than Julia's `Array` type.

The basic syntax for defining a composite type is

In [6]:
type Vector2D
    x::Float64
    y::Float64
end

The double colon (`::`) is a **type annotation** that specifies the type of `x` and `y`.

Let's  check which methods have been defined:

In [3]:
methods(Vector2D)

# 2 methods for generic function "Vector2D":
Vector2D(x::Float64,y::Float64)
Vector2D(x,y)

In [4]:
Vector2D(3.5, 4.5)

Vector2D(3.5,4.5)

In [7]:
Vector2D(3, 4)

Vector2D(3.0,4.0)

In [8]:
v = Vector2D(3, 4)
w = Vector2D(5, 6)

Vector2D(5.0,6.0)

What happens if we try to add two `Vector2D`s?

In [10]:
v + w

LoadError: `+` has no method matching +(::Vector2D, ::Vector2D)
while loading In[10], in expression starting on line 1

Julia confirms that it has no idea how to do so. So let's just try defining it:

In [11]:
+(a::Vector2D, b::Vector2D) = Vector2D(a.x+b.x, a.y+b.y)

+ (generic function with 118 methods)

In [12]:
v + w

Vector2D(8.0,10.0)

Suddenly, it works! This is Julia's version of operator overloading -- just add a new method to the relevant operator that acts on objects of your new type.

In [13]:
dot(a::Vector2D, b::Vector2D) = a.x*b.x + a.y+b.y

dot (generic function with 1 method)

Julia has certain Unicode operators defined, for example `\cdot<TAB>`:

In [15]:
⋅

dot (generic function with 7 methods)

We see that this operator is just an alias for the `dot` function, but that is parsed as in infix operator. So we can now write

In [16]:
v ⋅ w

LoadError: `dot` has no method matching dot(::Vector2D, ::Vector2D)
while loading In[16], in expression starting on line 1

It doesn't work. What's the matter? The signal was given by Julia when we defined `dot`: it said there was only 1 method. But `\cdot<TAB>` has 7 methods. So they are different functions. The solution is that the name `dot` refers to Julia's built-in `dot` function, which is defined in Julia's standard library, called `Base`. So the true name of the function is `Base.dot`:

In [18]:
Base.dot

dot (generic function with 7 methods)

We thus need to *extend* this function instead:

In [19]:
Base.dot(a::Vector2D, b::Vector2D) = a.x*b.x + a.y+b.y

dot (generic function with 8 methods)

In [20]:
v ⋅ w

25.0

Julia provides an `import` keyword to make this simpler when we are defining different methods for the function:

In [21]:
import Base.dot

This must be done *before* defining `dot`. To clear our current namespace we can use `workspace`:

In [22]:
workspace()

In [23]:
dot

dot (generic function with 8 methods)

In [24]:
type Vector2D
    x::Float64
    y::Float64
end

In [25]:
import Base.dot

In [26]:
dot(a::Vector2D, b::Vector2D) = a.x*b.x + a.y+b.y

dot (generic function with 9 methods)

In [27]:
v = Vector2D(1, 2)
w = Vector2D(3, 4)

Vector2D(3.0,4.0)

In [28]:
v ⋅ w

9.0

## Example: Automatic differentiation 

As in other languages, a type's behaviour is determined both by its fields and by the methods that act on the types; the only difference, once again, is that in Julia the methods are defined outside the definition of the type itself.

We will now define another type that has two fields but that behaves very differently, to permit [**automatic** (or **algorithmic**) **differentiation**](https://en.wikipedia.org/wiki/Automatic_differentiation).

The idea is that to differentiation a complicated function, say

$f(x) := \sin[(x-1)(x^2+2)],$

the function is decomposed into elementary functions that we know how to differentiate.

For example, suppose that we know how to differentiate the functions $g(x) := x-1$ and $h(x) := x^2 + 2$ at $x=a$. Then the derivative of their product is given by

$[g \cdot h]'(a) = g(a) h'(a) + g'(a) h(a)$.

Similarly, to differentiate *any* function at $a$, it is sufficient to know the value and the derivative of each subpart of the function at the same point $a$.

We thus represent a function $f$ by the pair $(f(a), f'(a))$, which we can regard as a mathematical object called a [**jet**](https://en.wikipedia.org/wiki/Jet_(mathematics)), i.e. the collection [equivalence class] of all functions that have the same value and the same derivative at the point $a$.

In [1]:
immutable Jet
    value::Float64
    deriv::Float64
end

The operations on `Jet`s should correspond to the operations on those functions to give new functions. For example, we have

In [2]:
+(f::Jet, g::Jet) = Jet(f.value+g.value, f.deriv+g.deriv)
*(f::Jet, g::Jet) = Jet(f.value*g.value, f.value*g.deriv + f.deriv*g.value)

* (generic function with 116 methods)

A constant function $c(x) := c$  for all $x$ has derivative $0$, so $c$ corresponds to the 

In [3]:
+(a::Jet, b::Number)  = +(promote(a, b)...)
convert(::Type{Jet}, c::Real) = Jet(c, 0)
promote_rule{S<:Real}(::Type{Jet}, ::Type{S}) = Jet

promote_rule (generic function with 1 method)

In [4]:
j = Jet(3, 1)

Jet(3.0,1.0)

In [5]:
j + 2

LoadError: stack overflow
while loading In[5], in expression starting on line 1

In [7]:
@which promote(j, 2)

promote{T,S}(x::T,y::S) at promotion.jl:123

In [8]:
promote_rule(Jet, Int)

Jet (constructor with 2 methods)

In [9]:
@which promote(j, 2)

promote{T,S}(x::T,y::S) at promotion.jl:123

In [9]:
@which promote_type(Jet, Int)

promote_type{T,S}(::Type{T},::Type{S}) at promotion.jl:110